In [2]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip

### State we should get

$$\rho' = \mathbb{I} + \delta \left[ \sum_\omega \gamma(\omega) \left(-\frac{1}{2} \{\sigma_x(\omega)^\dagger \sigma_x(\omega), \rho\} + \sigma_x(\omega) \rho \sigma_x(\omega)^\dagger\right) \right]+ \mathcal{O}(\delta^2)$$
Where we used the $\sigma_x$ jump on the 1st system qubit out of the [0, 1, 2]. Here it is still under inverstigation how large should $\delta$ be s.t we can really drop the higher orders, and also that we only have those terms there that should be dropped.

In [3]:
num_qubits = 3
num_energy_bits = 6
delta = 0.01
eps = 0.05
sigma = 10
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1


hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 10
step_size = total_time / num_trotter_steps
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, 
                                      coeffs=hamiltonian.rescaled_coeffs, shift=(total_time*hamiltonian.shift))

# U = expm(1j * total_time * hamiltonian.qt.full())
# print(np.linalg.norm(U - trott_U_qt.full()))

#* Initial state
with open(f'data/initial_state_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    initial_state = pickle.load(f)

initial_dm = DensityMatrix(initial_state).data

Original spectrum:  [-3.5    -2.9921 -2.3725 -1.5     1.6025  2.1     2.8895  3.7725]
Ideal spectrum:  [-0.0071  0.0255  0.0653  0.1214  0.3209  0.3529  0.4036  0.4604]
Nonrescaled coefficients:  [0.9 0.7 0.9 1. ]
Rescaled coefficients:  [0.05785714 0.045      0.05785714 0.06428571]
eXX, eYY, eZZ for qubit (0, 1)
eXX, eYY, eZZ for qubit (1, 2)
eXX, eYY, eZZ for qubit (2, 0)


In [4]:
boltzmann = lambda w: np.min([1, np.exp(-beta * w)])

time_labels = np.arange(-0.5, 0.5, 1/2**num_energy_bits)
energy_labels = 2*np.pi / (time_labels * 2**num_energy_bits)
energy_labels[int(2**num_energy_bits / 2)] = 0

jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()

dm_we_should_get = np.eye(2**num_qubits, dtype=np.complex128)
# for w in energy_labels:
#     oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, hamiltonian=hamiltonian.qt.full())
#     dm_we_should_get += boltzmann(w)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
#                                       -0.5 * initial_dm @ oft_op.conj().T @ oft_op
#                                       + oft_op @ initial_dm @ oft_op.conj().T)

for w in energy_labels:
    oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, U=trott_U_qt.full())
    dm_we_should_get += boltzmann(w)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
                                      -0.5 * initial_dm @ oft_op.conj().T @ oft_op
                                      + oft_op @ initial_dm @ oft_op.conj().T)
    


/var/folders/pg/6vn02n251m14stcv7vg03pjr0000gn/T/ipykernel_25725/2210737432.py:4: RuntimeWarning: divide by zero encountered in divide
  energy_labels = 2*np.pi / (time_labels * 2**num_energy_bits)


TypeError: exponent must be an integer

In [ ]:
the_time = 1
time_evolution_trott = lambda t: np.linalg.matrix_power(trott_U_qt, 1 * t)
time_evolution = lambda t: expm(1j * 2*np.pi * t * hamiltonian.qt.full())
np.linalg.norm(time_evolution(the_time) - time_evolution_trott(the_time))

0.09961190603426436

In [ ]:
with open(f'data/state_before_measure_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    circ_state = pickle.load(f)

circ_sys_dm = partial_trace(circ_state, list(range(2 + num_energy_bits))).data

In [ ]:
dist = np.linalg.norm(dm_we_should_get - circ_sys_dm)
dist

3.1329677184427442